In [1]:
# Célula 1: Importar bibliotecas
import pandas as pd
import numpy as np
import os
import sys
import plotly.express as px

In [2]:

# Adiciona o diretório 'src' ao caminho do sistema para que possamos importar nosso módulo
# Isso garante que o notebook encontre o db_connector.py
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.db_connector import get_db_connection, get_raw_data

print("Bibliotecas importadas e caminho do projeto configurado com sucesso.")


Bibliotecas importadas e caminho do projeto configurado com sucesso.


In [3]:
engine = get_db_connection()

if engine:
    print("Conexão com o banco de dados estabelecida.")

    with open('../sql/Estoque REF - GRI.sql', 'r') as file:
        query_estoque = file.read()

    df_estoque_raw = get_raw_data(query_estoque, engine)


    print(f"Dados de Estoque carregados: {len(df_estoque_raw)} linhas.")
    #print(f"Dados de Fechamento carregados: {len(df_fechamento_raw)} linhas.")
else:
    print("Falha na conexão com o banco de dados.")

df_estoque = df_estoque_raw.copy()



Conexão com o banco de dados estabelecida.
Dados de Estoque carregados: 6762 linhas.


In [4]:
engine = get_db_connection()

if engine:
    print("Conexão com o banco de dados estabelecida.")
    
    
    with open('../sql/Fechamento FKM.sql', 'r', encoding='utf-8') as file:
        query_fechamento = file.read()
        
    
    
    df_fechamento_raw = get_raw_data(query_fechamento, engine)
    
    print(f"Dados do fechamento FKM carregados: {len(df_fechamento_raw)} linhas.")
    
else:
    print("Falha na conexão com o banco de dados.")
    
df_fechamento = df_fechamento_raw.copy()

Conexão com o banco de dados estabelecida.
Dados do fechamento FKM carregados: 25492 linhas.


In [5]:
df_estoque.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6762 entries, 0 to 6761
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DataAtualizacaoDados  6762 non-null   datetime64[ns]
 1   IdVeiculo             6762 non-null   int64         
 2   Chassi                6762 non-null   object        
 3   Placa                 6762 non-null   object        
 4   RENAVAM               6614 non-null   object        
 5   IdCorVeiculo          6762 non-null   int64         
 6   Cor                   6762 non-null   object        
 7   AnoModelo             6762 non-null   int64         
 8   AnoFabricacao         6762 non-null   int64         
 9   IdModelo              6762 non-null   int64         
 10  Modelo                6762 non-null   object        
 11  IdMontadora           6762 non-null   int64         
 12  Montadora             6762 non-null   object        
 13  IdGrupoVeiculo    

In [6]:
df_fechamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25492 entries, 0 to 25491
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   IdNF                          25492 non-null  int64         
 1   NumeroNF                      25492 non-null  object        
 2   OrdemServico                  25492 non-null  object        
 3   Ocorrencia                    25492 non-null  object        
 4   OrdemCompra                   25492 non-null  object        
 5   Placa                         25492 non-null  object        
 6   IdVeiculo                     25492 non-null  int64         
 7   DescricaoItem                 25492 non-null  object        
 8   TipoItem                      25492 non-null  object        
 9   Tipo                          25492 non-null  object        
 10  TipoOrdemCompra               25492 non-null  object        
 11  IdGrupoDespesa              

In [7]:
# Célula de Tratamento de Datas no df_fechamento

# --- 1. Padronizar as colunas de data ---
# Primeiro, vamos garantir que a coluna 'DataCriacao' também seja do tipo datetime.
# Usar errors='coerce' é uma boa prática para evitar erros com dados inválidos.
df_fechamento['DataCriacao'] = pd.to_datetime(df_fechamento['DataCriacao'], errors='coerce')
df_fechamento['DataEntrada'] = pd.to_datetime(df_fechamento['DataEntrada'], errors='coerce')

# Verificar se alguma data falhou na conversão (opcional, mas bom para depuração)
datas_invalidas_criacao = df_fechamento['DataCriacao'].isnull().sum()
datas_invalidas_entrada = df_fechamento['DataEntrada'].isnull().sum()

if datas_invalidas_criacao > 0:
    print(f"Aviso: {datas_invalidas_criacao} valores na coluna 'DataCriacao' não puderam ser convertidos para data.")
if datas_invalidas_entrada > 0:
    print(f"Aviso: {datas_invalidas_entrada} valores na coluna 'DataEntrada' não puderam ser convertidos para data.")


# --- 2. Criar a coluna 'Mes/Ano' ---
# Sua lógica original está perfeita. Ela cria um objeto do tipo 'Period'.
df_fechamento['Mes/Ano'] = df_fechamento['DataCriacao'].dt.to_period('M')


# --- 3. (Opcional) Criar a coluna 'Mes/Ano' como texto ---
# Se, por algum motivo, você precisar do formato 'AAAA-MM' como texto (string), 
# você pode usar o método strftime. Isso é útil para alguns tipos de gráficos ou relatórios.
# df_fechamento['Mes/Ano_Texto'] = df_fechamento['DataCriacao'].dt.strftime('%Y-%m')


# --- 4. Visualizar o resultado ---
# Mostra as colunas relevantes para confirmar que o tratamento funcionou.
colunas_para_verificar = ['DataCriacao', 'DataEntrada', 'Mes/Ano']
print("\nTipos de dados após o tratamento:")
print(df_fechamento[colunas_para_verificar].info())

print("\nVisualização das colunas tratadas:")
print(df_fechamento[colunas_para_verificar].head())



Tipos de dados após o tratamento:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25492 entries, 0 to 25491
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DataCriacao  25492 non-null  datetime64[ns]
 1   DataEntrada  25492 non-null  datetime64[ns]
 2   Mes/Ano      25492 non-null  period[M]     
dtypes: datetime64[ns](2), period[M](1)
memory usage: 597.6 KB
None

Visualização das colunas tratadas:
              DataCriacao DataEntrada  Mes/Ano
0 2025-07-02 17:38:57.477  2025-07-01  2025-07
1 2025-07-02 17:38:57.957  2025-07-01  2025-07
2 2025-07-02 17:38:57.477  2025-07-01  2025-07
3 2025-07-02 17:38:57.477  2025-07-01  2025-07
4 2025-07-02 17:38:57.477  2025-07-01  2025-07


In [8]:
mapa_regiao_correto = {
    # --- MAPEAMENTO DA IMAGEM ORIGINAL ---
    'Gritsch Curitiba Base': 'SUL',
    'Gritsch Maringá': 'SUL',
    'Gritsch Florianópolis': 'SUL',
    'Gritsch Campo Grande': 'CENTRO OESTE',
    'Gritsch Blumenau': 'SUL',
    'Gritsch Cascavel': 'SUL',
    'Gritsch Porto Alegre': 'SUL',
    'Gritsch São Paulo (Perus)': 'SUDESTE',
    'Gritsch Goiânia': 'CENTRO OESTE',
    'Gritsch Brasília': 'CENTRO OESTE',
    'Gritsch Rondonópolis': 'CENTRO OESTE',
    'Gritsch São Paulo (Freguesia)': 'SUDESTE',
    'Gritsch Chapecó': 'SUL',
    'Gritsch Joinville': 'SUL',
    'Gritsch Londrina': 'SUL',
    'Gritsch Curitibanos': 'SUL',
    'Gritsch Criciuma': 'SUL',
    'Gritsch Sinop': 'CENTRO OESTE',
    'Gritsch Pato Branco': 'SUL',
    'Gritsch Cuiabá': 'CENTRO OESTE',
    'Gritsch Rio Verde': 'CENTRO OESTE',
    'Gritsch Laranjeiras do Sul': 'SUL',
    'Gritsch Itumbiara': 'CENTRO OESTE',
    'Gritsch Ponta Grossa': 'SUL',
    'Gritsch Curitiba ECT': 'SUL',
    'Gritsch Salvador': 'NORDESTE',
    'Gritsch Guarapuava': 'SUL',
    'Gritsch Palmas': 'NORTE',

    # --- MAPEAMENTO DA LISTA DE NÃO ENCONTRADOS ---
    # SUDESTE
    'REFERÊNCIA SÃO PAULO': 'SUDESTE',
    'REFERÊNCIA RIBEIRAO PRETO': 'SUDESTE',
    'GRITSCH - SAO (PERUS)': 'SUDESTE',
    'GRITSCH - SAO (FREGUESIA)': 'SUDESTE',
    'GRITSCH - MATRIZ': 'SUDESTE',
    'GRITSCH - PGR': 'SUL', # 
    'GRITSCH - ITR': 'CENTRO OESTE', 

    # SUL
    'REFERÊNCIA FOZ DO IGUAÇU': 'SUL',
    'REFERÊNCIA CURITIBA': 'SUL',
    'REFERÊNCIA MARINGÁ': 'SUL',
    'GRITSCH - CWB (BASE)': 'SUL',
    'GRITSCH - FLN': 'SUL',
    'GRITSCH - BLN': 'SUL',
    'GRITSCH - POA': 'SUL',
    'GRITSCH - LDB': 'SUL',
    'GRITSCH - CSC': 'SUL',
    'GRITSCH - CWB (ECT)': 'SUL',
    'GRITSCH - CHA': 'SUL',
    'GRITSCH - CRI': 'SUL',
    'GRITSCH - PBC': 'SUL',
    'GRITSCH - CTB': 'SUL',
    'GRITSCH - GPA': 'SUL',
    'GRITSCH - MGA': 'SUL',
    'GRITSCH - CWB (DIR)': 'SUL',
    'GRITSCH - JOI': 'SUL',

    # CENTRO-OESTE
    'REFERÊNCIA GOIÂNIA LOJA': 'CENTRO OESTE',
    'REFERÊNCIA BRASILIA': 'CENTRO OESTE',
    'REFERÊNCIA CUIABÁ LOJA': 'CENTRO OESTE',
    'REFERÊNCIA SINOP': 'CENTRO OESTE',
    'GRITSCH - SNO': 'CENTRO OESTE',
    'GRITSCH - RDN': 'CENTRO OESTE',
    'GRITSCH - GOI': 'CENTRO OESTE',
    'GRITSCH - CGB': 'CENTRO OESTE',
    'GRITSCH - CGR': 'CENTRO OESTE',
    'GRITSCH - RVD': 'CENTRO OESTE',
    'REFERÊNCIA GOIÂNIA AV RIO VERDE': 'CENTRO OESTE',
    'GRITSCH - BSB': 'CENTRO OESTE',
    'GRITSCH - CBL': 'CENTRO OESTE',

    # NORDESTE
    'REFERÊNCIA SALVADOR': 'NORDESTE',
    'REFERÊNCIA LUIS EDUARDO MAGALHÃES': 'NORDESTE',
    'REFERÊNCIA BALSAS': 'NORDESTE',
    'GRITSCH - SSA': 'NORDESTE',

    # NORTE
    'REFERÊNCIA ARAGUAÍNA': 'NORTE',
    'REFERÊNCIA VILHENA': 'NORTE',

    # RATEIOS (Assumidos como Sul/Matriz)
    'RATEIO - REF': 'RATEIO',
    'RATEIO GRI/REF': 'RATEIO',
    'RATEIO - GRI': 'RATEIO'

}


In [9]:
nome_coluna_filial = "FILIAL"

df_fechamento['Regiao'] = df_fechamento[nome_coluna_filial].map(mapa_regiao_correto)

filiais_nao_mapeadas = df_fechamento[df_fechamento['Regiao'].isnull()][nome_coluna_filial].unique()

if len (filiais_nao_mapeadas) >0:
    print("Filiais não mapeadas encontradas:")
    print(filiais_nao_mapeadas)
    
    df_fechamento['Regiao'] = df_fechamento['Regiao'].fillna('Regiao Nao definida', inplace=True)
    
else: 
    print("Todas as filiais foram mapeadas com sucesso.")

Todas as filiais foram mapeadas com sucesso.


In [12]:
# Célula: Criação da Coluna 'Empresa' para o Dashboard

import numpy as np
import pandas as pd

# Passo 1: Definir as condições e as escolhas para a criação da coluna 'Empresa'
# A ordem aqui não é crítica, mas é bom manter a consistência.
conditions = [
    (df_fechamento['FILIAL'].str.contains('GRITSCH', case=False, na=False)),
    (df_fechamento['FILIAL'].str.contains('REFERÊNCIA', case=False, na=False)),
    (df_fechamento['FILIAL'].str.contains('RATEIO', case=False, na=False)),
]

# Passo 2: Definir os valores correspondentes para cada condição
choices = [
    'Gritsch',
    'Referência',
    'Rateio',
]

# Passo 3: Usar np.select para criar a coluna 'Empresa'
# O 'default' será 'Outro' para qualquer filial que não se encaixe nas categorias.
df_fechamento['Empresa'] = np.select(conditions, choices, default='Outro')


# Passo 4: Verificar o resultado da criação da coluna
print("Coluna 'Empresa' criada com sucesso.")
print("\nDistribuição de registros por Empresa:")
print(df_fechamento['Empresa'].value_counts())


# Passo 5: Verificar se a lógica funcionou com exemplos específicos
print("\nAmostra dos dados para confirmar a lógica:")
amostra_gritsch = df_fechamento[df_fechamento['FILIAL'].str.contains('GRITSCH', na=False)].head(1)
amostra_referencia = df_fechamento[df_fechamento['FILIAL'].str.contains('REFERÊNCIA', na=False)].head(1)
amostra_rateio = df_fechamento[df_fechamento['FILIAL'].str.contains('RATEIO', na=False)].head(1)

amostra_final = pd.concat([
    amostra_gritsch, 
    amostra_referencia, 
    amostra_rateio
])

print(amostra_final[['FILIAL', 'Empresa']])


Coluna 'Empresa' criada com sucesso.

Distribuição de registros por Empresa:
Empresa
Referência    18898
Rateio         3728
Gritsch        2866
Name: count, dtype: int64

Amostra dos dados para confirmar a lógica:
                  FILIAL     Empresa
692        GRITSCH - SNO     Gritsch
0    REFERÊNCIA SALVADOR  Referência
14          RATEIO - REF      Rateio


In [13]:
# Célula: Análise de Custo por Empresa

import plotly.express as px

# Garantir que a coluna de valor não tem nulos
df_fechamento['ValorTotal'] = df_fechamento['ValorTotal'].fillna(0)

# 1. Agregando os dados por Empresa e Mês/Ano
df_custo_por_empresa = df_fechamento.groupby(['Mes/Ano', 'Empresa']).agg(
    CustoTotal=('ValorTotal', 'sum')
).reset_index()

# Remover linhas onde o custo é zero
df_custo_por_empresa = df_custo_por_empresa[df_custo_por_empresa['CustoTotal'] != 0]

# 2. Preparar a coluna de data para o gráfico
df_custo_por_empresa['Mes/Ano_str'] = df_custo_por_empresa['Mes/Ano'].astype(str)

# 3. Criar o gráfico
fig_por_empresa = px.bar(
    df_custo_por_empresa.sort_values('Mes/Ano_str'),
    x='Mes/Ano_str',
    y='CustoTotal',
    color='Empresa', # Usando a nova coluna 'Empresa'
    title='Custo Mensal por Empresa',
    labels={'Mes/Ano_str': 'Mês/Ano', 'CustoTotal': 'Custo Total (R$)'}
)
fig_por_empresa.show()
